# **Tweedie & TDBoost-Package**

In [44]:
import numpy as np
import scipy.special

In [45]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

## (1) Foundation

##### Exponential Family
- **EQ:** $$f_{X}(x|\theta) = h(x)\text{exp}[\eta(\theta)T(x)-A(\theta)]$$
- **FOUNDATION**
    
    - $\bold{X}$: **Random Variable**. The "abstract" thing thats being measured AND has NOT been observed yet.
        - Ex: "number of heads when you flip _ coins"
        - Ex: "height of a person in a given population"

    - $\bold{x}$: **Observed Value**. What has been actually *SEEN* or *MEASURED*. It's just one *POSSIBLE* outcome that $\bold{X}$ can be.
        - EX: "3 heads out of 5 coin flips"
        - EX: "someone's height measured to be 6ft tall"
    
    - $\bold{\theta}$: **Parameter of the Distribution**. Controls the shape of the distribution. *NOT* something we *DIRECTLY* observe. We typically try to *ESTIMATE* it from the given data. $\bold{\text{ex:}}\set{p, \mu, \lambda, ...}$

| Scenario           | \( X \) (Random Variable)              | \( x \) (Observed Value) | \( $\theta$ \) (Parameter)                  |
|--------------------|----------------------------------------|--------------------------|-------------------------------------------|
| Coin flips         | Number of heads in 10 flips            | 7                        | \( $\theta$ = p \) (probability of heads)   |
| Measuring height   | Height of a random person              | 172 cm                   | \( $\theta$ = $\mu$ \) (average height)       |
| Waiting time       | Time between bus arrivals              | 5.2 minutes              | \( $\theta$ = $\lambda$ \) (rate of arrival)  |
| Rainfall on a day  | Rain amount (mm)                       | 0 mm                     | \( $\theta$ = $\mu$ \) (mean rainfall)        |

- **EXPLANATIONS**

    - $\bold{f_{X}(x|\theta)}$: Probability of seeing a particular $\bold{x}$ value given some value of $\bold{\theta}$.

    - $\bold{h(x)}$: **Base function**. (1) It normalize the functions so that the area under the curve is equal to 1. (2) This part of the equation adjusts the shape of the function in the final output. **(only a function of x)**
    
    - $\bold{\text{exp}[...]}$: **>:|** its in the name!

    - $\bold{\eta(\theta)}$: **Natural Parameter**. The idea is to convert the original distribution for $\theta$ into a form that makes it compatible for the exponential family distribution.
        - It can be thought as putting your parameter ($\set{p, \mu, \lambda, ...}$) through a translator ($\eta(?)$) to make it compatible with the exponential family equation. 

    - $\bold{T(x)}$: **Sufficient Statistic**: Basically the minimum amount of information & a *"transformation"* of $\bold{x}$ that converts it to a more suitable form.
        - Ex:
            | x (Data) | T(x) | Explanation |
            | -------- | ---- | ----------- |
            | 0 or 1   | $T(x)$ = x | outcome of trail is enough |
            | [0,1,1,0,1] | $T(x) = \set{\text{count(1)... so 3}}$ | Sum of successes |
            | 4 | $T(x) = x$ | Value itself |
            | [160, 170, 180] | $T(x) = \frac{\sum x}{\text{number of vals}} $ | average value |

    - $\bold{A(\theta)}$: **Log-Partition Function**: Normalizes the parameter value ($\set{p, \mu, \lambda, ...}$) so that the final function has an area of 1.


## (1.1) Playing Around


| Distribution       | $h(x)$                                           | $\eta(\theta)$                                 | $T(x)$                                 | $A(\theta)$                                           | Domain                          |
|--------------------|--------------------------------------------------|--------------------------------------------------|------------------------------------------|--------------------------------------------------------|----------------------------------|
| **Normal** (known variance) | $\frac{1}{\sqrt{2\pi}} e^{-x^2/2}$              | $\mu$                                            | $x$                                      | $\frac{\mu^2}{2}$                                     | $\mathbb{R}$                     |
| **Exponential**    | $1$                                              | $-\lambda$                                       | $x$                                      | $-\log(-\eta)$                                       | $[0, \infty)$                    |
| **Gamma** (fixed shape $k$) | $\frac{x^{k-1}}{\Gamma(k)}$                   | $-\beta$                                         | $x$                                      | $-k \log(-\eta)$                                      | $(0, \infty)$                    |
| **Chi-squared**    | Like Gamma with $k = \frac{v}{2}$                | $-\frac{1}{2}$                                   | $x$                                      | $\frac{v}{2} \log 2 + \log \Gamma(v/2)$               | $(0, \infty)$                    |
| **Beta**           | $1$                                              | $(\alpha - 1, \beta - 1)$                        | $(\log x, \log(1 - x))$                  | $\log B(\alpha, \beta)$                              | $(0, 1)$                         |
| **Dirichlet**      | $1$                                              | $\alpha_i - 1$ (vector)                          | $\log x_i$                               | $\log B(\boldsymbol{\alpha})$                        | $\{x_i > 0, \sum x_i = 1\}$      |
| **Bernoulli**      | $1$                                              | $\log\left(\frac{p}{1-p}\right)$                | $x$                                      | $\log(1 + e^\eta)$                                   | $\{0, 1\}$                       |
| **Categorical**    | $1$                                              | $\log p_i$                                       | $x_i$ (indicator)                        | $\log \sum_i e^{\eta_i}$                             | Discrete over $K$ classes       |
| **Poisson**        | $\frac{1}{x!}$                                   | $\log \lambda$                                   | $x$                                      | $e^{\eta}$                                           | $\mathbb{N}_0$                  |
| **Wishart**        | $\propto |\Sigma|^{-(\nu + p + 1)/2} \exp\left(-\frac{1}{2} \text{tr}(\Sigma^{-1} X)\right)$ | $-\frac{1}{2}\Sigma^{-1}$      | $X$                                      | Normalizing constant (det, trace, gamma)             | PSD matrices                    |
| **Inverse Wishart**| Similar to Wishart but inverted                  | Complex (non-canonical form)                     | $X^{-1}$                                 | Involves determinant and trace                        | PSD matrices                    |
| **Geometric**      | $1$                                              | $\log(1 - p)$                                    | $x$                                      | $-\log(1 - e^{\eta})$                                | $\mathbb{N}_0$                  |


In [46]:
def make_plots(f, x, theta):
    fig = make_subplots(
        rows=3,
        cols=2,
        #shared_xaxes=True,
        subplot_titles=["h(x)", "n(theta)", "T(x)", "A(theta)", "f(x|theta)=h(x)exp[n(theta)T(x)-A(theta)]"]
    )
    
    fig.add_trace(
        trace= go.Scatter(
            x= x,
            y= f.h(x),
            name= "h(x)"
        ),
        row=1,
        col=1
    )
    fig.add_trace(
        trace= go.Scatter(
            x= theta,
            y= f.n(theta),
            name= "n(theta)"
        ),
        row=1,
        col=2
    )
    fig.add_trace(
        trace= go.Scatter(
            x= x,
            y= f.T(x),
            name= "T(x)"
        ),
        row=2,
        col=1
    )
    fig.add_trace(
        trace= go.Scatter(
            x= theta,
            y= f.A(x),
            name= "A(theta)"
        ),
        row=2,
        col=2
    )
    fig.add_trace(
        trace= go.Scatter(
            x= x,
            y= f.eq(x, theta[0]),
            name= "f(x|theta)=h(x)exp[n(theta)T(x)-A(theta)]"
        ),
        row=3,
        col=1
    )
    fig.update_xaxes(title_text="x", row=1, col=1)
    fig.update_xaxes(title_text="theta", row=1, col=2)
    fig.update_xaxes(title_text="x", row=2, col=1)
    fig.update_xaxes(title_text="theta", row=2, col=2)
    fig.update_xaxes(title_text="x", row=3, col=1)

    fig.update_yaxes(title_text="h(x)", row=1, col=1)
    fig.update_yaxes(title_text="n(theta)", row=1, col=2)
    fig.update_yaxes(title_text="T(x)", row=2, col=1)
    fig.update_yaxes(title_text="A(theta)", row=2, col=2)
    fig.update_yaxes(title_text="f(x|theta)", row=3, col=1)
    
    steps = []
    for theta_val in theta:
        step = dict(
            method="update",
            args=[
                {
                    "y": [
                        f.h(x),
                        [f.n(t) for t in theta],
                        f.T(x),
                        [f.A(t) for t in theta],
                        f.eq(x, theta_val)
                    ]
                    },
                {"title": f"theta = {theta_val:.2f}"}],
            label=f"{theta_val:.2f}"
        )
        steps.append(step)

    sliders = [dict(
        active=len(theta) // 2,
        currentvalue={"prefix": "theta: "},
        pad={"t": 50},
        steps=steps
    )]

    fig.update_layout(
        height=1000,
        width=800*2,
        sliders=sliders,
        title=f"Exponential Family Components for theta = {theta[0]:.2f}"
    )

    fig.show()

| Distribution       | $h(x)$                                           | $\eta(\mu)$                                 | $T(x)$                                 | $A(\mu)$                                           | Domain                          |
|--------------------|--------------------------------------------------|--------------------------------------------------|------------------------------------------|--------------------------------------------------------|----------------------------------|
| **Normal** (known variance) | $\frac{1}{\sqrt{2\pi}} e^{-x^2/2}$              | $\mu$                                            | $x$                                      | $\frac{\mu^2}{2}$                                     | $\mathbb{R}$                     |

In [47]:
class Normal:
    
    @staticmethod
    def h(x):
        return (1/(np.sqrt(2*np.pi)))*np.exp((-1/2)*np.power(x, 2))
    
    @staticmethod
    def n(mu):
        return mu
    
    @staticmethod
    def T(x):
        return x
    
    @staticmethod
    def A(mu):
        A_mu = np.power(mu, 2)*(0.5)
        return A_mu
    
    def eq(self, x, mu):
        return self.h(x)*np.exp(
            self.n(mu)*self.T(x) - self.A(mu)
        )

In [48]:
F = Normal()
x = np.linspace(
    start= -10,
    stop= 10,
    num= 1_000
)
mu = np.linspace(
    start= -10,
    stop= 10,
    num= 1_000
)

In [49]:
NORMAL_PLOTS= False

In [50]:
if NORMAL_PLOTS:
    make_plots(
        f= F,
        x= x,
        theta= mu
    )

| Distribution       | $h(x)$                                           | $\eta(\lambda)$                                 | $T(x)$                                 | $A(\lambda)$                                           | Domain                          |
|--------------------|--------------------------------------------------|--------------------------------------------------|------------------------------------------|--------------------------------------------------------|----------------------------------|
| **Poisson**        | $\frac{1}{x!}$                                   | $\log \lambda$                                   | $x$                                      | $\lambda$                                           | $\mathbb{N}_0$                  |


In [51]:
import scipy.special

class Poisson:
    
    @staticmethod
    def h(x):
        return (1/scipy.special.factorial(x))
    
    @staticmethod
    def n(theta):
        return np.log(theta)
    
    @staticmethod
    def T(x):
        return x
    
    @staticmethod
    def A(theta):
        return theta
    
    def eq(self, x, mu):
        return self.h(x)*np.exp(
            self.n(mu)*self.T(x) - self.A(mu)
        )

| Distribution       | $h(x)$                                           | $\eta(\lambda)$                                 | $T(x)$                                 | $A(\lambda)$                                           | Domain                          |
|--------------------|--------------------------------------------------|--------------------------------------------------|------------------------------------------|--------------------------------------------------------|----------------------------------|
| **Tweedie**          |


In [52]:
F = Poisson()
START = 1
STOP = 100

x = np.array(
    object= [i for i in range(START, STOP+1)],
    dtype= np.int64
)
theta = np.linspace(
    start= START,
    stop= STOP,
    num= 100
)

In [53]:
POISSON_PLOTS = False

In [54]:
if POISSON_PLOTS:
    make_plots(
        f= F,
        x= x,
        theta= theta
    )

In [55]:
class Gamma:
    
    @staticmethod
    def h(x):
        return  (1/scipy.special.gamma(x))*np.power(x, 0.5 - 1)
    
    @staticmethod
    def n(theta):
        return -theta
    
    @staticmethod
    def T(x):
        return x
    
    @staticmethod
    def A(theta):
        return (-0.5)*np.log(theta)
    
    def eq(self, x, mu):
        return self.h(x)*np.exp(
            self.n(mu)*self.T(x) - self.A(mu)
        )

In [56]:
F = Gamma()

x = np.linspace(
    start= 0.1,
    stop= 10,
    num= 1_000
)
theta = np.linspace(
    start= 0.1,
    stop= 5,
    num= 1_000
)

In [57]:
GAMMA_PLOTS = False

In [58]:
if GAMMA_PLOTS:
    make_plots(
        f= F,
        x= x,
        theta= theta
    )

### Exploring Tweedie Distribution

| Distribution  | $h(x)$                                                                                       | $\eta(\lambda)$               | $T(x)$ | $A(\lambda)$                 | Domain                     |
|---------------|----------------------------------------------------------------------------------------------|-------------------------------|--------|------------------------------|----------------------------|
| **Tweedie**   | $\displaystyle \frac{1}{x}\,W_{-\alpha,0}\!\Bigl(-\frac{x}{\phi(p-1)}\Bigr),\;\alpha=\frac{2-p}{p-1}$ | $\displaystyle \frac{\lambda^{1-p}}{1-p}$ | $x$    | $\displaystyle \frac{\lambda^{2-p}}{2-p}$ | $\{0\}\cup(0,\infty)$ |


In [59]:
class Tweedie:
    @staticmethod
    def h(x, phi, p):
        alpha = (2 - p) / (p - 1)
        return (1/x) * scipy.special.wright_bessel(-alpha, 0, -x/(phi*(p-1)))
    @staticmethod
    def n(mu, p):
        return mu**(1 - p) / (1 - p)
    @staticmethod
    def T(x):         return x
    @staticmethod
    def A(mu, p):      return mu**(2 - p) / (2 - p)
    def eq(self, x, mu, phi, p):
        return self.h(x,phi,p) * np.exp((self.n(mu,p)*self.T(x) - self.A(mu,p)) / phi)

| Parameter  | Domain                   |
|------------|--------------------------|
| $x$        | $[0,\infty)$  |
| $\mu$  | $(0,\infty)$     |
| $\phi$     | $(0,\infty)$         |
| $p$        | $(1,2)$                |


In [60]:
F = Tweedie()

x = np.linspace(
    start= 0.1,
    stop= 10,
    num= 1_000
)

mu = np.linspace(
    start= 0.1,
    stop= 10,
    num= 1_000
)

phi = np.linspace(
    start= 0.1,
    stop= 10,
    num= 1_000
)

p = np.linspace(
    start= 1,
    stop= 2,
    num= 1_000
)